In [1]:
import pandas as pd
import numpy as np
import seaborn as sbs

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 120)
print("Versions:",
      "pandas", pd.__version__,
      "| numpy", np.__version__)

Versions: pandas 2.3.3 | numpy 2.3.3


In [2]:
df = sbs.load_dataset("titanic")
df.head(5)

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [3]:
df.info()
df.isna().mean().sort_values(ascending=False).to_frame("missing_rate")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB


,missing_rate
deck,0.772166
age,0.198653
embarked,0.002245
embark_town,0.002245
survived,0.000000
pclass,0.000000
sex,0.000000
sibsp,0.000000
parch,0.000000
fare,0.000000


In [4]:
keep = ["survived", "pclass", "sex", "age", "sibsp", "parch", "fare", "embarked", "alone"]
df =df[keep].copy()
df.head(5)

,survived,pclass,sex,age,sibsp,parch,fare,embarked,alone
0,0,3,male,22.0,1,0,7.2500,S,False
1,1,1,female,38.0,1,0,71.2833,C,False
2,1,3,female,26.0,0,0,7.9250,S,True
3,1,1,female,35.0,1,0,53.1000,S,False
4,0,3,male,35.0,0,0,8.0500,S,True


In [5]:
age_medians = df.groupby(["sex", "pclass"])["age"].median()
def impute_age(row):
    if pd.isna(row["age"]):
        return age_medians.loc[(row["sex"], row["pclass"])]
    return row["age"]

df["age"] = df.apply(impute_age, axis=1)

embarked_mode = df["embarked"].mode(dropna=True)[0]
df["embarked"] = df["embarked"].fillna(embarked_mode)

df["fare"] = df["fare"].fillna(df["fare"].median())

df.isna().sum()

survived    0
pclass      0
sex         0
age         0
sibsp       0
parch       0
fare        0
embarked    0
alone       0
dtype: int64

In [6]:
df.describe(include="all")
df.dtypes

survived      int64
pclass        int64
sex          object
age         float64
sibsp         int64
parch         int64
fare        float64
embarked     object
alone          bool
dtype: object

In [7]:
from pathlib import Path
Path("data").mkdir(exist_ok=True)
df.to_csv("data/titanic_clean.csv", index=False)
len(df), "rows saved to data/titanic_clean.csv"

(891, 'rows saved to data/titanic_clean.csv')

In [8]:
from sklearn.model_selection import train_test_split

# Separate features and target
X = df.drop("survived", axis=1)
y = df["survived"]

# One-hot encode categorical variables
# Machine-learning models (like logistic regression or random forest) cannot process text or categorical data directly.
# pd.get_dummies() creates new binary columns (0/1) for each category (a process called one-hot encoding).
X = pd.get_dummies(X, drop_first=True)

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Sanity check
X_train.shape, X_test.shape, y_train.mean(), y_test.mean()

((712, 9),
 (179, 9),
 np.float64(0.38342696629213485),
 np.float64(0.3854748603351955))